In [47]:
# Imports necesarios para todos los experimentos de Feature Engineering
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.manifold import TSNE
from sklearn.preprocessing import PolynomialFeatures
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
import json
warnings.filterwarnings('ignore')
# Diccionario global para almacenar resultados de todos los experimentos
global_results = {}

In [48]:
df = pd.read_csv("../../datasets/transaction_dataset_clean.csv")
X = df.drop(columns=["FLAG"])
y = df["FLAG"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Experimento Baseline: Evaluación con características originales
    
Este experimento establece la línea base usando las características originales sin ninguna transformación o ingeniería de features. Sirve como punto de referencia para comparar todos los demás experimentos.

In [49]:
def experiment_baseline(X_train, X_test, y_train, y_test):
    """ 
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados del experimento baseline
    """
    print("📊 Experimento Baseline: Características originales")
    print("-" * 50)
    
    # Crear experimento con características originales
    experiments = {
        'original_features': (X_train, X_test)
    }
    
    # Evaluar experimento baseline
    results = evaluate_multiple_experiments(experiments, "baseline", y_train, y_test)
    
    print(f"\nBaseline establecido con {X_train.shape[1]} características originales")
    print(f"Accuracy baseline: {results['original_features']['accuracy']:.4f}")
    
    return results

### Experimento 1: Características basadas en árboles de decisión
    
Este experimento utiliza modelos de árboles (XGBoost y Random Forest) para generar nuevas características basadas en:
- Índices de hojas donde terminan las muestras
- Combinaciones de múltiples modelos de árboles
- Estadísticas derivadas de las estructuras de árboles
    
Los modelos de árboles capturan patrones no lineales complejos y sus estructuras internas pueden usarse como características transformadas.
    

In [50]:
def experiment_tree_based_features(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("🌳 Experimento 1: Características basadas en árboles")
    print("-" * 50)
    
    # 1.1 XGBoost leaf indices
    # Entrena un modelo XGBoost y extrae los índices de las hojas donde
    # terminan las muestras. Cada árbol contribuye con un índice de hoja.
    print("Generando características con XGBoost leaf indices...")
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # apply() devuelve los índices de hojas para cada muestra en cada árbol
    leaf_indices_train = xgb_model.apply(X_train)
    leaf_indices_test = xgb_model.apply(X_test)
    
    # 1.2 Random Forest leaf indices
    # Similar a XGBoost pero usando Random Forest, que puede capturar
    # patrones diferentes debido a su estrategia de bagging
    print("Generando características con Random Forest leaf indices...")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # apply() devuelve los índices de hojas para cada árbol del bosque
    rf_leaf_train = rf_model.apply(X_train)
    rf_leaf_test = rf_model.apply(X_test)
    
    # 1.3 Combinación de ambos modelos
    # Concatena las características de ambos modelos para capturar
    # la información complementaria de XGBoost y Random Forest
    print("Combinando características de XGBoost y Random Forest...")
    combined_leaves_train = np.hstack([leaf_indices_train, rf_leaf_train])
    combined_leaves_test = np.hstack([leaf_indices_test, rf_leaf_test])
    
    # 1.4 Estadísticas derivadas: distancias promedio a hojas
    # Calcula el promedio de los índices de hojas como una medida
    # de la "profundidad promedio" o "complejidad" de la decisión
    print("Calculando estadísticas derivadas de hojas...")
    leaf_distances_train = np.mean(leaf_indices_train, axis=1).reshape(-1, 1)
    leaf_distances_test = np.mean(leaf_indices_test, axis=1).reshape(-1, 1)
    
    # Preparar experimentos para evaluación
    experiments = {
        'xgb_leaves': (leaf_indices_train, leaf_indices_test),
        'rf_leaves': (rf_leaf_train, rf_leaf_test),
        'combined_leaves': (combined_leaves_train, combined_leaves_test),
        'leaf_distances': (leaf_distances_train, leaf_distances_test)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "tree_based", y_train, y_test)
    
    print(f"\nMejor resultado en tree-based features:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Experimento 2: Reducción de dimensionalidad
    
Este experimento aplica diferentes técnicas de reducción de dimensionalidad para crear representaciones más compactas y potencialmente más informativas de los datos:
    
- PCA: Encuentra componentes principales que maximizan la varianza
- TruncatedSVD: Descomposición en valores singulares, útil para datos sparse
- FastICA: Análisis de componentes independientes, separa señales mezcladas
- Combinación: Usa características originales + PCA para enriquecimiento
    

In [51]:
def experiment_dimensionality_reduction(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("📉 Experimento 2: Reducción de dimensionalidad")
    print("-" * 50)
    
    # 2.1 PCA (Principal Component Analysis)
    # Encuentra las direcciones de máxima varianza en los datos
    # Útil para eliminar ruido y correlaciones lineales
    print("Aplicando PCA...")
    n_components_pca = min(50, X_train.shape[1])
    pca = PCA(n_components=n_components_pca)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    print(f"  PCA: {X_train.shape[1]} → {n_components_pca} componentes")
    print(f"  Varianza explicada: {pca.explained_variance_ratio_[:5].sum():.3f} (primeros 5 componentes)")
    
    # 2.2 TruncatedSVD (Singular Value Decomposition)
    # Similar a PCA pero no centra los datos, mejor para matrices sparse
    # Útil cuando los datos tienen muchos ceros o están normalizados
    print("Aplicando TruncatedSVD...")
    n_components_svd = min(50, X_train.shape[1]-1)
    svd = TruncatedSVD(n_components=n_components_svd, random_state=42)
    X_train_svd = svd.fit_transform(X_train)
    X_test_svd = svd.transform(X_test)
    
    print(f"  SVD: {X_train.shape[1]} → {n_components_svd} componentes")
    print(f"  Varianza explicada: {svd.explained_variance_ratio_[:5].sum():.3f} (primeros 5 componentes)")
    
    # 2.3 FastICA (Independent Component Analysis)
    # Separa señales mezcladas asumiendo independencia estadística
    # Útil para encontrar fuentes de variación independientes
    print("Aplicando FastICA...")
    n_components_ica = min(20, X_train.shape[1])
    ica = FastICA(n_components=n_components_ica, random_state=42)
    X_train_ica = ica.fit_transform(X_train)
    X_test_ica = ica.transform(X_test)
    
    print(f"  ICA: {X_train.shape[1]} → {n_components_ica} componentes independientes")
    
    # 2.4 Combinación: Características originales + PCA
    # Mantiene la información original y añade componentes principales
    # Estrategia de enriquecimiento que puede capturar tanto patrones
    # lineales (PCA) como no lineales (características originales)
    print("Combinando características originales con PCA...")
    X_train_combined = np.hstack([X_train, X_train_pca])
    X_test_combined = np.hstack([X_test, X_test_pca])
    
    print(f"  Combinado: {X_train.shape[1]} + {n_components_pca} = {X_train_combined.shape[1]} características")
    
    # Preparar experimentos para evaluación
    experiments = {
        'pca_only': (X_train_pca, X_test_pca),
        'svd_only': (X_train_svd, X_test_svd),
        'ica_only': (X_train_ica, X_test_ica),
        'original_plus_pca': (X_train_combined, X_test_combined)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "dimensionality_reduction", y_train, y_test)
    
    print(f"\nMejor resultado en reducción de dimensionalidad:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Experimento 3: Interacciones entre características
    
Este experimento crea nuevas características basadas en las interacciones entre las características existentes:
    
- Características polinómicas: Productos entre pares de características
- Interacciones manuales: Productos selectivos entre características principales
- Ratios: Divisiones entre características para capturar proporciones
    
Las interacciones pueden revelar patrones que no son evidentes cuando se consideran las características individualmente.
    

In [52]:
def experiment_feature_interactions(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("🔄 Experimento 3: Interacciones entre características")
    print("-" * 50)
    
    # 3.1 Características polinómicas (grado 2)
    # Genera automáticamente todos los productos entre pares de características
    # include_bias=False: no incluye término constante
    # interaction_only=True: solo productos cruzados, no cuadrados
    print("Generando características polinómicas...")
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    
    print(f"  Polinómicas: {X_train.shape[1]} → {X_train_poly.shape[1]} características")
    
    # 3.2 Interacciones manuales entre características principales
    # Selecciona las primeras N características y genera productos entre ellas
    # Más controlado que PolynomialFeatures, evita explosión de características
    print("Generando interacciones manuales...")
    n_features = min(10, X_train.shape[1])
    interactions_train = []
    interactions_test = []
    
    interaction_count = 0
    for i in range(n_features):
        for j in range(i+1, n_features):
            # Producto entre características i y j
            interaction_train = (X_train.iloc[:, i] * X_train.iloc[:, j]).values
            interaction_test = (X_test.iloc[:, i] * X_test.iloc[:, j]).values
            
            interactions_train.append(interaction_train)
            interactions_test.append(interaction_test)
            interaction_count += 1
    
    X_train_interactions = np.column_stack(interactions_train)
    X_test_interactions = np.column_stack(interactions_test)
    
    print(f"  Interacciones manuales: {interaction_count} nuevas características")
    
    # 3.3 Ratios entre características
    # Calcula ratios (divisiones) entre características para capturar proporciones
    # Útil para detectar patrones relativos entre variables
    print("Generando ratios entre características...")
    ratios_train = []
    ratios_test = []
    
    ratio_count = 0
    n_features_ratio = min(5, X_train.shape[1])
    
    for i in range(n_features_ratio):
        for j in range(i+1, n_features_ratio):
            # Evitar división por cero agregando pequeña constante
            denominator_train = X_train.iloc[:, j] + 1e-8
            denominator_test = X_test.iloc[:, j] + 1e-8
            
            # Ratio entre características i y j
            ratio_train = (X_train.iloc[:, i] / denominator_train).values
            ratio_test = (X_test.iloc[:, i] / denominator_test).values
            
            ratios_train.append(ratio_train)
            ratios_test.append(ratio_test)
            ratio_count += 1
    
    X_train_ratios = np.column_stack(ratios_train)
    X_test_ratios = np.column_stack(ratios_test)
    
    print(f"  Ratios: {ratio_count} nuevas características")
    
    # Preparar experimentos para evaluación
    experiments = {
        'polynomial_features': (X_train_poly, X_test_poly),
        'manual_interactions': (X_train_interactions, X_test_interactions),
        'feature_ratios': (X_train_ratios, X_test_ratios)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "feature_interactions", y_train, y_test)
    
    print(f"\nMejor resultado en interacciones de características:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Experimento 4: Características basadas en clustering
    
Este experimento utiliza técnicas de clustering para crear nuevas características basadas en la estructura de agrupamiento de los datos:
    
- Etiquetas de cluster: Asignación de muestras a grupos
- Distancias a centroides: Qué tan lejos está cada muestra de los centros
- Clustering por clase: Distancias a centroides específicos de cada clase
    
El clustering puede revelar patrones de agrupamiento natural en los datos que pueden ser útiles para la clasificación.
    

In [53]:
def experiment_clustering_features(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("🎯 Experimento 4: Características basadas en clustering")
    print("-" * 50)
    
    # 4.1 KMeans clustering básico
    # Agrupa los datos en k clusters y usa las etiquetas como características
    print("Aplicando K-Means clustering...")
    n_clusters = 10
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels_train = kmeans.fit_predict(X_train)
    cluster_labels_test = kmeans.predict(X_test)
    
    print(f"  K-Means: {n_clusters} clusters creados")
    print(f"  Distribución de clusters en entrenamiento: {np.bincount(cluster_labels_train)}")
    
    # 4.2 Distancias a centroides de K-Means
    # Calcula la distancia de cada muestra a cada centroide
    # Proporciona información más rica que solo las etiquetas
    print("Calculando distancias a centroides...")
    distances_train = kmeans.transform(X_train)
    distances_test = kmeans.transform(X_test)
    
    print(f"  Distancias: {distances_train.shape[1]} características de distancia")
    
    # 4.3 Clustering específico por clase
    # Calcula centroides separados para cada clase y mide distancias
    # Útil para detectar qué tan similar es una muestra a cada clase
    print("Calculando centroides por clase...")
    
    # Separar datos por clase
    fraud_mask = (y_train == 1)
    normal_mask = (y_train == 0)
    
    fraud_data = X_train[fraud_mask]
    normal_data = X_train[normal_mask]
    
    print(f"  Datos de fraude: {fraud_data.shape[0]} muestras")
    print(f"  Datos normales: {normal_data.shape[0]} muestras")
    
    # Calcular centroides por clase
    fraud_centroid = fraud_data.mean().values
    normal_centroid = normal_data.mean().values
    
    # Distancias a centroide de clase fraudulenta
    fraud_dist_train = np.linalg.norm(X_train - fraud_centroid, axis=1).reshape(-1, 1)
    fraud_dist_test = np.linalg.norm(X_test - fraud_centroid, axis=1).reshape(-1, 1)
    
    # Distancias a centroide de clase normal
    normal_dist_train = np.linalg.norm(X_train - normal_centroid, axis=1).reshape(-1, 1)
    normal_dist_test = np.linalg.norm(X_test - normal_centroid, axis=1).reshape(-1, 1)
    
    # Combinar distancias a ambas clases
    class_distances_train = np.hstack([fraud_dist_train, normal_dist_train])
    class_distances_test = np.hstack([fraud_dist_test, normal_dist_test])
    
    print(f"  Distancias por clase: 2 características (dist_fraud, dist_normal)")
    
    # Estadísticas adicionales de distancias
    print(f"  Distancia promedio a centroide fraude: {np.mean(fraud_dist_train):.3f}")
    print(f"  Distancia promedio a centroide normal: {np.mean(normal_dist_train):.3f}")
    
    # Preparar experimentos para evaluación
    experiments = {
        'kmeans_labels': (cluster_labels_train.reshape(-1, 1), cluster_labels_test.reshape(-1, 1)),
        'kmeans_distances': (distances_train, distances_test),
        'class_distances': (class_distances_train, class_distances_test)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "clustering", y_train, y_test)
    
    print(f"\nMejor resultado en características de clustering:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Experimento 5: Características de detección de anomalías
    
Este experimento utiliza técnicas de detección de anomalías para crear características que midan qué tan "anómala" o "inusual" es cada muestra:
    
- Isolation Forest: Aísla anomalías usando árboles aleatorios
- Distancia de Mahalanobis: Distancia considerando covarianza
- Estadísticas locales: Z-scores y medidas de desviación
    
Estas características son especialmente útiles para detección de fraude, ya que los fraudes suelen ser anomalías en los patrones normales.
    

In [54]:
def experiment_anomaly_features(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("🚨 Experimento 5: Características de detección de anomalías")
    print("-" * 50)
    
    # 5.1 Isolation Forest scores
    # Isolation Forest aísla anomalías usando árboles de decisión aleatorios
    # Los puntos anómalos requieren menos divisiones para ser aislados
    print("Aplicando Isolation Forest...")
    iso_forest = IsolationForest(n_estimators=100, random_state=42)
    iso_forest.fit(X_train)
    
    # decision_function devuelve scores de anomalía (más negativo = más anómalo)
    anomaly_scores_train = iso_forest.decision_function(X_train).reshape(-1, 1)
    anomaly_scores_test = iso_forest.decision_function(X_test).reshape(-1, 1)
    
    print(f"  Isolation Forest: 1 característica de anomalía")
    print(f"  Score promedio entrenamiento: {np.mean(anomaly_scores_train):.3f}")
    print(f"  Score promedio test: {np.mean(anomaly_scores_test):.3f}")
    
    # 5.2 Distancia de Mahalanobis
    # Mide distancia considerando la covarianza entre características
    # Más robusta que distancia euclidiana para datos correlacionados
    print("Calculando distancias de Mahalanobis...")
    
    # Calcular matriz de covarianza y su inversa
    cov_matrix = np.cov(X_train.T)
    
    # Usar pseudoinversa para manejar matrices singulares
    inv_cov = np.linalg.pinv(cov_matrix)
    mean_vector = np.mean(X_train, axis=0)
    
    def mahalanobis_distance(X, mean, inv_cov):
        """
        Calcula la distancia de Mahalanobis para cada muestra
        """
        diff = X - mean
        return np.sqrt(np.sum(diff @ inv_cov * diff, axis=1))
    
    # Calcular distancias de Mahalanobis
    mahal_dist_train = np.array(mahalanobis_distance(X_train, mean_vector, inv_cov)).reshape(-1, 1)
    mahal_dist_test = np.array(mahalanobis_distance(X_test, mean_vector, inv_cov)).reshape(-1, 1)

    print(f"  Mahalanobis: 1 característica de distancia")
    print(f"  Distancia promedio entrenamiento: {np.mean(mahal_dist_train):.3f}")
    print(f"  Distancia promedio test: {np.mean(mahal_dist_test):.3f}")
    
    # 5.3 Estadísticas locales por característica
    # Calcula Z-scores para cada característica individual
    # Identifica muestras que son outliers en características específicas
    print("Calculando estadísticas locales (Z-scores)...")
    local_stats_train = []
    local_stats_test = []
    
    for col in X_train.columns:
        # Calcular media y desviación estándar de entrenamiento
        col_mean = X_train[col].mean()
        col_std = X_train[col].std()
        
        # Z-score: (valor - media) / desviación estándar
        z_score_train = (X_train[col] - col_mean) / col_std
        z_score_test = (X_test[col] - col_mean) / col_std
        
        local_stats_train.append(z_score_train.values)
        local_stats_test.append(z_score_test.values)
    
    local_stats_train = np.column_stack(local_stats_train)
    local_stats_test = np.column_stack(local_stats_test)
    
    print(f"  Estadísticas locales: {local_stats_train.shape[1]} características Z-score")
    
    # 5.4 Estadísticas agregadas de anomalía
    # Combina múltiples medidas de anomalía en características únicas
    print("Creando estadísticas agregadas...")
    
    # Máximo Z-score absoluto por muestra (outlier más extremo)
    max_zscore_train = np.max(np.abs(local_stats_train), axis=1).reshape(-1, 1)
    max_zscore_test = np.max(np.abs(local_stats_test), axis=1).reshape(-1, 1)
    
    # Número de características con |Z-score| > 2 (outliers moderados)
    outlier_count_train = np.sum(np.abs(local_stats_train) > 2, axis=1).reshape(-1, 1)
    outlier_count_test = np.sum(np.abs(local_stats_test) > 2, axis=1).reshape(-1, 1)
    
    # Combinar estadísticas agregadas
    aggregated_stats_train = np.hstack([max_zscore_train, outlier_count_train])
    aggregated_stats_test = np.hstack([max_zscore_test, outlier_count_test])
    
    print(f"  Estadísticas agregadas: {aggregated_stats_train.shape[1]} características")
    
    # Preparar experimentos para evaluación
    experiments = {
        'isolation_forest': (anomaly_scores_train, anomaly_scores_test),
        'mahalanobis_distance': (mahal_dist_train, mahal_dist_test),
        'local_statistics': (local_stats_train, local_stats_test),
        'aggregated_stats': (aggregated_stats_train, aggregated_stats_test)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "anomaly_detection", y_train, y_test)
    
    print(f"\nMejor resultado en detección de anomalías:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Experimento 6: Selección de características
    
Este experimento utiliza diferentes técnicas de selección de características para identificar y mantener solo las más relevantes:
    
- Mutual Information: Mide dependencia estadística entre características y target
- F-score: Estadístico F de ANOVA para clasificación
- RFE: Eliminación recursiva usando Random Forest
- SelectFromModel: Selección basada en importancia de XGBoost
    
La selección de características puede mejorar el rendimiento eliminando ruido y características irrelevantes, y reduce el riesgo de overfitting.
    

In [55]:
def experiment_feature_selection(X_train, X_test, y_train, y_test):
    """
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones del experimento
    """
    print("🎯 Experimento 6: Selección de características")
    print("-" * 50)
    
    # 6.1 Mutual Information
    # Mide la dependencia estadística entre cada característica y el target
    # Captura relaciones tanto lineales como no lineales
    print("Aplicando selección por Mutual Information...")
    k_features_mi = min(20, X_train.shape[1])
    
    selector_mi = SelectKBest(score_func=mutual_info_classif, k=k_features_mi)
    X_train_mi = selector_mi.fit_transform(X_train, y_train)
    X_test_mi = selector_mi.transform(X_test)
    
    # Obtener scores de mutual information
    mi_scores = selector_mi.scores_
    selected_features_mi = X_train.columns[selector_mi.get_support()]
    
    print(f"  Mutual Information: {X_train.shape[1]} → {k_features_mi} características")
    print(f"  Score promedio seleccionadas: {np.mean(mi_scores[selector_mi.get_support()]):.3f}")
    print(f"  Top 3 características: {selected_features_mi[:3].tolist()}")
    
    # 6.2 F-score (ANOVA F-test)
    # Estadístico F que mide la diferencia entre medias de grupos
    # Efectivo para características con relaciones lineales con el target
    print("Aplicando selección por F-score...")
    k_features_f = min(20, X_train.shape[1])
    
    selector_f = SelectKBest(score_func=f_classif, k=k_features_f)
    X_train_f = selector_f.fit_transform(X_train, y_train)
    X_test_f = selector_f.transform(X_test)
    
    # Obtener F-scores
    f_scores = selector_f.scores_
    selected_features_f = X_train.columns[selector_f.get_support()]
    
    print(f"  F-score: {X_train.shape[1]} → {k_features_f} características")
    print(f"  F-score promedio seleccionadas: {np.mean(f_scores[selector_f.get_support()]):.3f}")
    print(f"  Top 3 características: {selected_features_f[:3].tolist()}")
    
    # 6.3 RFE (Recursive Feature Elimination) con Random Forest
    # Elimina características recursivamente basándose en importancia del modelo
    # Considera interacciones entre características
    print("Aplicando RFE con Random Forest...")
    n_features_rfe = min(15, X_train.shape[1])
    
    # Usar Random Forest como estimador base
    rf_estimator = RandomForestClassifier(n_estimators=50, random_state=42)
    rf_selector = RFE(rf_estimator, n_features_to_select=n_features_rfe)
    
    X_train_rfe = rf_selector.fit_transform(X_train, y_train)
    X_test_rfe = rf_selector.transform(X_test)
    
    # Obtener características seleccionadas
    selected_features_rfe = X_train.columns[rf_selector.get_support()]
    feature_rankings = rf_selector.ranking_
    
    print(f"  RFE: {X_train.shape[1]} → {n_features_rfe} características")
    print(f"  Características seleccionadas: {selected_features_rfe[:3].tolist()}")
    
    # 6.4 SelectFromModel con XGBoost
    # Selecciona características basándose en importancia de XGBoost
    # Automáticamente determina el umbral óptimo
    print("Aplicando SelectFromModel con XGBoost...")
    
    # Entrenar XGBoost para obtener importancias
    xgb_estimator = xgb.XGBClassifier(n_estimators=50, random_state=42)
    xgb_selector = SelectFromModel(xgb_estimator)
    
    X_train_xgb = xgb_selector.fit_transform(X_train, y_train)
    X_test_xgb = xgb_selector.transform(X_test)
    
    # Obtener información sobre la selección
    selected_features_xgb = X_train.columns[xgb_selector.get_support()]
    feature_importances = xgb_selector.estimator_.feature_importances_
    
    print(f"  XGBoost: {X_train.shape[1]} → {X_train_xgb.shape[1]} características")
    print(f"  Umbral automático: {xgb_selector.threshold_:.4f}")
    print(f"  Top 3 características: {selected_features_xgb[:3].tolist()}")
    
    # 6.5 Análisis de consenso
    # Identifica características seleccionadas por múltiples métodos
    print("Analizando consenso entre métodos...")
    
    all_selected = set(selected_features_mi) | set(selected_features_f) | \
                  set(selected_features_rfe) | set(selected_features_xgb)
    
    consensus_features = set(selected_features_mi) & set(selected_features_f) & \
                        set(selected_features_rfe) & set(selected_features_xgb)
    
    print(f"  Características únicas total: {len(all_selected)}")
    print(f"  Características en consenso: {len(consensus_features)}")
    if len(consensus_features) > 0:
        print(f"  Consenso: {list(consensus_features)[:5]}")
    
    # Preparar experimentos para evaluación
    experiments = {
        'mutual_info_selection': (X_train_mi, X_test_mi),
        'f_score_selection': (X_train_f, X_test_f),
        'rfe_selection': (X_train_rfe, X_test_rfe),
        'xgb_selection': (X_train_xgb, X_test_xgb)
    }
    
    # Evaluar todas las variaciones
    results = evaluate_multiple_experiments(experiments, "feature_selection", y_train, y_test)
    
    print(f"\nMejor resultado en selección de características:")
    best_method = max(results.items(), key=lambda x: x[1]['accuracy'])
    print(f"  {best_method[0]}: {best_method[1]['accuracy']:.4f}")
    
    return results

### Funciones de evaluación, resumen y ejecución de experimentos

En este bloque se agrupan las funciones principales para la gestión de los experimentos de ingeniería de características:

- **evaluate_experiment:** Evalúa un conjunto de características transformadas usando un modelo de regresión logística y calcula el accuracy.
- **evaluate_multiple_experiments:** Permite evaluar varias variantes de un mismo tipo de experimento y organiza los resultados en el diccionario global.
- **print_experiment_summary:** Muestra un resumen de los resultados de todos los experimentos de ingeniería de características, incluyendo ranking, estadísticas y recomendaciones.
- **run_all_experiments:** Ejecuta en secuencia todos los experimentos de feature engineering, almacena los resultados globales y los guarda en un archivo JSON para su posterior análisis.

In [56]:
def evaluate_experiment(X_train_exp, X_test_exp, y_train, y_test, experiment_name):
    """
    Función auxiliar para evaluar un experimento específico usando Regresión Logística
    
    Parámetros:
    - X_train_exp: Datos de entrenamiento transformados
    - X_test_exp: Datos de test transformados  
    - y_train: Etiquetas de entrenamiento
    - y_test: Etiquetas de test
    - experiment_name: Nombre del experimento para mostrar en resultados
    
    Retorna:
    - Diccionario con accuracy, número de features y objetos entrenados
    """
    try:
        # Escalado de características usando StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_exp)
        X_test_scaled = scaler.transform(X_test_exp)
        
        # Entrenamiento del modelo de Regresión Logística
        lr = LogisticRegression(max_iter=1000, random_state=42)
        lr.fit(X_train_scaled, y_train)
        
        # Predicción y evaluación
        y_pred = lr.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        
        result = {
            'accuracy': accuracy,
            'n_features': X_train_exp.shape[1],
            'model': lr,
            'scaler': scaler
        }
        
        print(f"  {experiment_name}: {accuracy:.4f} (features: {X_train_exp.shape[1]})")
        return result
        
    except Exception as e:
        print(f"  {experiment_name}: Error - {str(e)}")
        return {'accuracy': 0, 'error': str(e)}

def evaluate_multiple_experiments(experiments, experiment_type, y_train, y_test):
    """
    Función auxiliar para evaluar múltiples variaciones de un tipo de experimento
    
    Parámetros:
    - experiments: Diccionario con nombre_experimento: (X_train, X_test)
    - experiment_type: Tipo de experimento (para organizar resultados)
    - y_train: Etiquetas de entrenamiento
    - y_test: Etiquetas de test
    
    Retorna:
    - Diccionario con resultados de todas las variaciones
    """
    results = {}
    
    for name, (X_train_exp, X_test_exp) in experiments.items():
        result = evaluate_experiment(X_train_exp, X_test_exp, y_train, y_test, name)
        results[name] = result
    
    global_results[experiment_type] = results
    return results

def print_experiment_summary():
    """
    Imprime un resumen completo de todos los experimentos ejecutados
    
    Analiza los resultados almacenados en global_results y muestra:
    - Ranking de todos los métodos por accuracy
    - Comparación de número de características
    - Mejores métodos por categoría
    - Estadísticas generales de los experimentos
    """
    print("\n" + "="*80)
    print("📋 RESUMEN COMPLETO DE EXPERIMENTOS DE FEATURE ENGINEERING")
    print("="*80)
    
    if not global_results:
        print("No se han ejecutado experimentos aún.")
        return
    
    # Recopilar todos los resultados
    all_results = []
    for exp_type, results in global_results.items():
        for method_name, result in results.items():
            if 'accuracy' in result and result['accuracy'] > 0:
                all_results.append({
                    'experiment': exp_type,
                    'method': method_name,
                    'accuracy': result['accuracy'],
                    'n_features': result.get('n_features', 0)
                })
    
    if not all_results:
        print("No hay resultados válidos para mostrar.")
        return
    
    # Ordenar por accuracy descendente
    all_results.sort(key=lambda x: x['accuracy'], reverse=True)
    
    # Mostrar ranking general
    print(f"\n🏆 RANKING GENERAL (Top 15)")
    print("-" * 80)
    print(f"{'Rank':<5} {'Experiment':<20} {'Method':<25} {'Accuracy':<10} {'Features':<8}")
    print("-" * 80)
    
    for i, result in enumerate(all_results[:15], 1):
        print(f"{i:<5} {result['experiment']:<20} {result['method']:<25} "
              f"{result['accuracy']:.4f}    {result['n_features']:<8}")
    
    # Estadísticas por categoría
    print(f"\n📊 MEJORES MÉTODOS POR CATEGORÍA")
    print("-" * 80)
    
    categories = {}
    for result in all_results:
        exp_type = result['experiment']
        if exp_type not in categories:
            categories[exp_type] = []
        categories[exp_type].append(result)
    
    for category, results in categories.items():
        best_result = max(results, key=lambda x: x['accuracy'])
        print(f"{category:<20}: {best_result['method']:<25} "
              f"({best_result['accuracy']:.4f}, {best_result['n_features']} features)")
    
    # Estadísticas generales
    print(f"\n📈 ESTADÍSTICAS GENERALES")
    print("-" * 80)
    
    accuracies = [r['accuracy'] for r in all_results]
    feature_counts = [r['n_features'] for r in all_results]
    
    print(f"Total de experimentos ejecutados: {len(all_results)}")
    print(f"Accuracy promedio: {np.mean(accuracies):.4f}")
    print(f"Accuracy máxima: {np.max(accuracies):.4f}")
    print(f"Accuracy mínima: {np.min(accuracies):.4f}")
    print(f"Desviación estándar: {np.std(accuracies):.4f}")
    print(f"Número promedio de características: {np.mean(feature_counts):.1f}")
    print(f"Rango de características: {np.min(feature_counts)} - {np.max(feature_counts)}")
    
    # Análisis de eficiencia (accuracy vs número de características)
    print(f"\n⚡ ANÁLISIS DE EFICIENCIA")
    print("-" * 80)
    
    # Encontrar métodos con alta accuracy y pocas características
    efficient_methods = [r for r in all_results if r['accuracy'] > np.mean(accuracies) 
                        and r['n_features'] <= np.mean(feature_counts)]
    
    if efficient_methods:
        print("Métodos eficientes (alta accuracy, pocas características):")
        for method in efficient_methods[:5]:
            efficiency_score = method['accuracy'] / (method['n_features'] + 1)
            print(f"  {method['method']:<25}: {method['accuracy']:.4f} "
                  f"({method['n_features']} features, score: {efficiency_score:.4f})")
    
    # Recomendaciones
    print(f"\n💡 RECOMENDACIONES")
    print("-" * 80)
    
    best_overall = all_results[0]
    print(f"🥇 Mejor método general: {best_overall['method']}")
    print(f"   Categoría: {best_overall['experiment']}")
    print(f"   Accuracy: {best_overall['accuracy']:.4f}")
    print(f"   Características: {best_overall['n_features']}")
    
    if efficient_methods:
        best_efficient = max(efficient_methods, key=lambda x: x['accuracy'])
        print(f"🚀 Mejor método eficiente: {best_efficient['method']}")
        print(f"   Accuracy: {best_efficient['accuracy']:.4f}")
        print(f"   Características: {best_efficient['n_features']}")

def run_all_experiments(X_train, X_test, y_train, y_test):
    """
    Ejecuta todos los experimentos de Feature Engineering en secuencia
    
    Esta función ejecuta sistemáticamente todos los experimentos disponibles:
    1. Baseline con características originales
    2. Características basadas en árboles
    3. Reducción de dimensionalidad
    4. Interacciones entre características
    5. Características de clustering
    6. Características de detección de anomalías
    7. Selección de características
    
    Parámetros:
    - X_train: DataFrame con características de entrenamiento
    - X_test: DataFrame con características de test
    - y_train: Series con etiquetas de entrenamiento
    - y_test: Series con etiquetas de test
    
    Retorna:
    - Diccionario con todos los resultados organizados por experimento
    """
    print("🔬 INICIANDO SUITE COMPLETA DE EXPERIMENTOS")
    print("="*80)
    print(f"Datos de entrenamiento: {X_train.shape}")
    print(f"Datos de test: {X_test.shape}")
    print(f"Distribución de clases: {np.bincount(y_train)}")
    print("="*80)
    
    # Limpiar resultados globales
    global_results.clear()
    
    # 1. Experimento Baseline
    print(f"\n{'='*20} EXPERIMENTO 0: BASELINE {'='*20}")
    baseline_results = experiment_baseline(X_train, X_test, y_train, y_test)
    
    # 2. Experimento de árboles
    print(f"\n{'='*20} EXPERIMENTO 1: ÁRBOLES {'='*20}")
    tree_results = experiment_tree_based_features(X_train, X_test, y_train, y_test)
    
    # 3. Experimento de reducción de dimensionalidad
    print(f"\n{'='*20} EXPERIMENTO 2: DIMENSIONALIDAD {'='*20}")
    dim_results = experiment_dimensionality_reduction(X_train, X_test, y_train, y_test)
    
    # 4. Experimento de interacciones
    print(f"\n{'='*20} EXPERIMENTO 3: INTERACCIONES {'='*20}")
    interaction_results = experiment_feature_interactions(X_train, X_test, y_train, y_test)
    
    # 5. Experimento de clustering
    print(f"\n{'='*20} EXPERIMENTO 4: CLUSTERING {'='*20}")
    clustering_results = experiment_clustering_features(X_train, X_test, y_train, y_test)
    
    # 6. Experimento de detección de anomalías
    print(f"\n{'='*20} EXPERIMENTO 5: ANOMALÍAS {'='*20}")
    anomaly_results = experiment_anomaly_features(X_train, X_test, y_train, y_test)
    
    # 7. Experimento de selección de características
    print(f"\n{'='*20} EXPERIMENTO 6: SELECCIÓN {'='*20}")
    selection_results = experiment_feature_selection(X_train, X_test, y_train, y_test)
    
    # Mostrar resumen final
    print_experiment_summary()
    
    # Guardar resultados en archivo JSON
    output_path = "feature_engineering_results.json"
    serializable_results = {
        exp: {
            method: {
                k: float(v) if isinstance(v, (np.floating, np.float32, np.float64)) else v
                for k, v in result.items() if k in ['accuracy', 'n_features']
            }
            for method, result in methods.items()
        }
        for exp, methods in global_results.items()
    }
    with open(output_path, "w") as f:
        json.dump(serializable_results, f, indent=2)
    print(f"\nResultados guardados en {output_path}")
    
    return global_results

def get_best_features(X_train, X_test, y_train, y_test, method_name=None):
    """
    Obtiene las mejores características transformadas basándose en los resultados
    
    Parámetros:
    - X_train, X_test, y_train, y_test: Datos originales
    - method_name: Nombre específico del método (opcional)
    
    Retorna:
    - Tupla con (X_train_transformed, X_test_transformed) del mejor método
    """
    if not global_results:
        print("No hay resultados disponibles. Ejecuta los experimentos primero.")
        return None, None
    
    # Encontrar el mejor método
    best_accuracy = 0
    best_method = None
    best_experiment = None
    
    for exp_type, results in global_results.items():
        for method, result in results.items():
            if method_name and method != method_name:
                continue
            if result.get('accuracy', 0) > best_accuracy:
                best_accuracy = result['accuracy']
                best_method = method
                best_experiment = exp_type
    
    if best_method is None:
        print("No se encontró el método especificado.")
        return None, None
    
    print(f"Aplicando mejor método: {best_method} (accuracy: {best_accuracy:.4f})")
    
    # Reejecutar el experimento específico para obtener las características
    if best_experiment == "baseline":
        return X_train, X_test
    elif best_experiment == "tree_based":
        results = experiment_tree_based_features(X_train, X_test, y_train, y_test)
    elif best_experiment == "dimensionality_reduction":
        results = experiment_dimensionality_reduction(X_train, X_test, y_train, y_test)
    elif best_experiment == "feature_interactions":
        results = experiment_feature_interactions(X_train, X_test, y_train, y_test)
    elif best_experiment == "clustering":
        results = experiment_clustering_features(X_train, X_test, y_train, y_test)
    elif best_experiment == "anomaly_detection":
        results = experiment_anomaly_features(X_train, X_test, y_train, y_test)
    elif best_experiment == "feature_selection":
        results = experiment_feature_selection(X_train, X_test, y_train, y_test)
    
    # Nota: Esta función devuelve None porque requeriría reejecutar
    # los experimentos para obtener las características transformadas
    print("Para obtener las características transformadas, reejecutar el experimento específico.")
    return None, None


### Ejecución de los experimentos


In [57]:
# Experimento baseline
baseline_results = experiment_baseline(X_train, X_test, y_train, y_test)

📊 Experimento Baseline: Características originales
--------------------------------------------------
  original_features: 0.8207 (features: 32)

Baseline establecido con 32 características originales
Accuracy baseline: 0.8207


In [58]:
# Experimento árboles
tree_results = experiment_tree_based_features(X_train, X_test, y_train, y_test)

🌳 Experimento 1: Características basadas en árboles
--------------------------------------------------
Generando características con XGBoost leaf indices...
Generando características con Random Forest leaf indices...
Combinando características de XGBoost y Random Forest...
Calculando estadísticas derivadas de hojas...
  xgb_leaves: 0.9827 (features: 100)
  rf_leaves: 0.9680 (features: 100)
  combined_leaves: 0.9827 (features: 200)
  leaf_distances: 0.7776 (features: 1)

Mejor resultado en tree-based features:
  xgb_leaves: 0.9827


In [59]:
# Experimento dimensionalidad
dim_results = experiment_dimensionality_reduction(X_train, X_test, y_train, y_test)

📉 Experimento 2: Reducción de dimensionalidad
--------------------------------------------------
Aplicando PCA...
  PCA: 32 → 32 componentes
  Varianza explicada: 1.000 (primeros 5 componentes)
Aplicando TruncatedSVD...
  SVD: 32 → 31 componentes
  Varianza explicada: 1.000 (primeros 5 componentes)
Aplicando FastICA...
  ICA: 32 → 20 componentes independientes
Combinando características originales con PCA...
  Combinado: 32 + 32 = 64 características
  pca_only: 0.8156 (features: 32)
  svd_only: 0.8187 (features: 31)
  ica_only: 0.7816 (features: 20)
  original_plus_pca: 0.8492 (features: 64)

Mejor resultado en reducción de dimensionalidad:
  original_plus_pca: 0.8492


In [60]:
# Experimento interacciones
interaction_results = experiment_feature_interactions(X_train, X_test, y_train, y_test)

🔄 Experimento 3: Interacciones entre características
--------------------------------------------------
Generando características polinómicas...
  Polinómicas: 32 → 528 características
Generando interacciones manuales...
  Interacciones manuales: 45 nuevas características
Generando ratios entre características...
  Ratios: 10 nuevas características
  polynomial_features: 0.8883 (features: 528)
  manual_interactions: 0.7918 (features: 45)
  feature_ratios: 0.7816 (features: 10)

Mejor resultado en interacciones de características:
  polynomial_features: 0.8883


In [61]:
# Experimento clustering
clustering_results = experiment_clustering_features(X_train, X_test, y_train, y_test)

🎯 Experimento 4: Características basadas en clustering
--------------------------------------------------
Aplicando K-Means clustering...
  K-Means: 10 clusters creados
  Distribución de clusters en entrenamiento: [7139    1    1    1   27    1  682    2    7   11]
Calculando distancias a centroides...
  Distancias: 10 características de distancia
Calculando centroides por clase...
  Datos de fraude: 1752 muestras
  Datos normales: 6120 muestras
  Distancias por clase: 2 características (dist_fraud, dist_normal)
  Distancia promedio a centroide fraude: 217445747.372
  Distancia promedio a centroide normal: 313206800.770
  kmeans_labels: 0.8603 (features: 1)
  kmeans_distances: 0.7831 (features: 10)
  class_distances: 0.7831 (features: 2)

Mejor resultado en características de clustering:
  kmeans_labels: 0.8603


In [62]:
# Experimento anomalías
anomaly_results = experiment_anomaly_features(X_train, X_test, y_train, y_test)

🚨 Experimento 5: Características de detección de anomalías
--------------------------------------------------
Aplicando Isolation Forest...
  Isolation Forest: 1 característica de anomalía
  Score promedio entrenamiento: 0.152
  Score promedio test: 0.153
Calculando distancias de Mahalanobis...
  Mahalanobis: 1 característica de distancia
  Distancia promedio entrenamiento: 0.000
  Distancia promedio test: 0.000
Calculando estadísticas locales (Z-scores)...
  Estadísticas locales: 32 características Z-score
Creando estadísticas agregadas...
  Estadísticas agregadas: 2 características
  isolation_forest: 0.7760 (features: 1)
  mahalanobis_distance: 0.7831 (features: 1)
  local_statistics: Error - Input X contains infinity or a value too large for dtype('float64').
  aggregated_stats: Error - Input X contains infinity or a value too large for dtype('float64').

Mejor resultado en detección de anomalías:
  mahalanobis_distance: 0.7831


In [63]:
# Experimento selección
selection_results = experiment_feature_selection(X_train, X_test, y_train, y_test)

🎯 Experimento 6: Selección de características
--------------------------------------------------
Aplicando selección por Mutual Information...
  Mutual Information: 32 → 20 características
  Score promedio seleccionadas: 0.197
  Top 3 características: ['Time Diff between first and last (Mins)', 'Received Tnx', 'Unique Received From Addresses']
Aplicando selección por F-score...
  F-score: 32 → 20 características
  F-score promedio seleccionadas: 50.712
  Top 3 características: ['Avg min between sent tnx', 'Avg min between received tnx', 'Time Diff between first and last (Mins)']
Aplicando RFE con Random Forest...
  RFE: 32 → 15 características
  Características seleccionadas: ['Avg min between received tnx', 'Time Diff between first and last (Mins)', 'Unique Received From Addresses']
Aplicando SelectFromModel con XGBoost...
  XGBoost: 32 → 5 características
  Umbral automático: 0.0312
  Top 3 características: ['Time Diff between first and last (Mins)', 'total ether received', 'Total ER

In [64]:
# Resumen final
print_experiment_summary()


📋 RESUMEN COMPLETO DE EXPERIMENTOS DE FEATURE ENGINEERING

🏆 RANKING GENERAL (Top 15)
--------------------------------------------------------------------------------
Rank  Experiment           Method                    Accuracy   Features
--------------------------------------------------------------------------------
1     tree_based           xgb_leaves                0.9827    100     
2     tree_based           combined_leaves           0.9827    200     
3     tree_based           rf_leaves                 0.9680    100     
4     feature_interactions polynomial_features       0.8883    528     
5     clustering           kmeans_labels             0.8603    1       
6     dimensionality_reduction original_plus_pca         0.8492    64      
7     baseline             original_features         0.8207    32      
8     dimensionality_reduction svd_only                  0.8187    31      
9     dimensionality_reduction pca_only                  0.8156    32      
10    feature_sele

In [65]:

# Ejecutar todo de una vez 
all_results = run_all_experiments(X_train, X_test, y_train, y_test)

🔬 INICIANDO SUITE COMPLETA DE EXPERIMENTOS
Datos de entrenamiento: (7872, 32)
Datos de test: (1969, 32)
Distribución de clases: [6120 1752]

==================== EXPERIMENTO 0: BASELINE ====================
📊 Experimento Baseline: Características originales
--------------------------------------------------
  original_features: 0.8207 (features: 32)

Baseline establecido con 32 características originales
Accuracy baseline: 0.8207

==================== EXPERIMENTO 1: ÁRBOLES ====================
🌳 Experimento 1: Características basadas en árboles
--------------------------------------------------
Generando características con XGBoost leaf indices...
Generando características con Random Forest leaf indices...
Combinando características de XGBoost y Random Forest...
Calculando estadísticas derivadas de hojas...
  xgb_leaves: 0.9827 (features: 100)
  rf_leaves: 0.9680 (features: 100)
  combined_leaves: 0.9827 (features: 200)
  leaf_distances: 0.7776 (features: 1)

Mejor resultado en tree-b